#### import libraries

In [1]:
import numpy as np

from astropy.table import Table

from grizli.aws import db
from grizli import utils
import eazy.utils

# import functions in dja_match.py file
import dja_match as dm

Set ROOT_PATH=/usr/local/share/python/dawn-dja-match


#### download basic info of unmatched spectra from AWS database (from table 'nirspec_extractions')

In [2]:
# Remove cosmos-transients-v3 from nirspec_phot_match so that the script has something to do while testing
if (1):
    db.execute("""delete from nirspec_phot_match where root_spec = 'cosmos-transients-v3'""")

In [3]:
ne_tab = dm.unmatched_spec_from_db()

In [4]:
print(f'N={len(ne_tab)} unmatched spectra')
# Summarize missing data by root
utils.Unique(ne_tab['root'])

N=2980 unmatched spectra
   N  value     
====  ==========
   1  glazebrook-cos-obs3-v2
   2  macsj0647-hr-v2
   5  rubies-uds1-v2
   5  ceers-ddt-v1
   7  glazebrook-v1
   7  glazebrook-v2
  12  rubies-egs61-v2
  12  ceers-ddt-v2
  13  rubies-egs1-v2
  18  macsj0647-v2
  18  macsj0647-v1
  20  smacs0723-ero-v2
  20  smacs0723-ero-v1
  21  macsj0647-single-v1
  23  macs1423-v1
  23  macs1423-v2
  25  abell370-v2
  26  abell370-v1
  27  macs0417-v1
  27  macs0417-v2
  27  macs0416-v2
  28  macs0416-v1
  29  rxj2129-ddt-v1
  30  borg-2203p1851-v1
  32  snh0pe-v2 
  33  borg-2203p1851-v2
  34  macs1149-v2
  35  borg-0859p4114-v1
  36  snH0pe-v1 
  38  borg-1033p5051-v1
  39  borg-1033p5051-v2
  40  borg-0859p4114-v2
  43  borg-1437p5044-v2
  45  suspense-kriek-v2
  45  borg-0314m6712-v1
  46  jades-gds-wide-lr-v1
  46  borg-1437p5044-v1
  50  borg-0314m6712-v2
  77  rxj2129-ddt-v2
  86  cecilia-v1
  86  cecilia-v2
  90  stark-a1703-v2
  96  jades-gds-wide-mr-v1
 112  whl0137-v1
 124  cosm

#### download basic info of photometric files from AWS database (from table 'grizli_photometry')

In [5]:
gr_tab = dm.grizli_phot_from_db()

In [6]:
print(f'N={len(gr_tab)} rows from grizli_photometry')

N=1159719 rows from grizli_photometry


#### match the spectra and photometry via [ra,dec]

In [7]:
# saves outfile matched catalogue to 'cats' folder
matched_tab = dm.match_spec_phot_cats(photcat_concat=gr_tab,
                                      spec_tab=ne_tab,
                                      sep=30, # Match separation
                                      outfile_name=None) #'spec_phot_match_tab'

In [8]:
has_match = matched_tab['dr'] < 0.25
print(f'Found {has_match.sum()} new matches')

Found 365 new matches


#### read output matched table 

(not used since the function above now just returns the table rather than writing to a file)

In [9]:
# if matched_tab is None:
#     matched_tab_read = utils.read_catalog('cats/spec_phot_match_tab.fits')
# else:
#     matched_tab_read = matched_tab

## Prepare the table for sending to the database

In [10]:
# Rename root > root_spec to avoid conflict with existing tables
if 'root' in matched_tab.colnames:
    matched_tab.rename_column('root', 'root_spec')

# Make sure columns are strings not bytes
for c in matched_tab.colnames:
    if isinstance(matched_tab[c][0], str):
        print('Set string column: ', c)
        matched_tab[c] = [str(r) for r in matched_tab[c]]

matched_tab['file_phot'] = [f.replace('phot_apcorr','phot') for f in matched_tab['file_phot']]

Set string column:  root_spec
Set string column:  file_spec
Set string column:  file_phot


In [11]:
# Photometric catalogs already done
done = db.SQL("""select file_phot, count(file_phot) from nirspec_phot_match
group by file_phot""")

In [12]:
# Photometric catalogs in matches
un = utils.Unique(matched_tab['file_phot'], verbose=True)

   N  value     
====  ==========
 247  primer-cosmos-west-grizli-v7.0-fix_phot.fits
 249  primer-cosmos-east-grizli-v7.0-fix_phot.fits


In [13]:
print(f"{'field':>50} {'new':>5} {'done':>5} {'new=done'}")

for v in un.values:
    if v in done['file_phot']:
        ix = done['file_phot'] == v
        nd = done['count'][ix][0]
        print(f"{v:>50} {un[v].sum():>5} {nd:>5}  {un[v].sum() == nd}")

                                             field   new  done new=done
      primer-cosmos-east-grizli-v7.0-fix_phot.fits   249   804  False
      primer-cosmos-west-grizli-v7.0-fix_phot.fits   247  1646  False


In [14]:
prev = np.isin(un.values, done['file_phot'])
print('New fields: \n-----------\n' + '\n'.join(np.array(un.values)[~prev].tolist()))

New fields: 
-----------



In [15]:
# Send rows grouped by file_phot
un = utils.Unique(matched_tab['file_phot'], verbose=False)

# Just test when False
SEND_TO_DB = True

for v in un.values:

    old = db.SQL(f"""select file_phot, file_spec from nirspec_phot_match where file_phot = '{v}'""")
    old_count = db.SQL(f"""select count(file_phot) from nirspec_phot_match where file_phot = '{v}'""")['count'][0]

    new = un[v] & (~np.isin(matched_tab['file_spec'], old['file_spec']))
    old = un[v] & (np.isin(matched_tab['file_spec'], old['file_spec']))

    print(f"Add {new.sum():>5} rows ({old.sum():>4} in matched_tab, {old_count:>4} in table)) to nirspec_phot_match for file_phot = '{v}'")

    # Now send the subset of rows
    if (old.sum() == old_count):
        # Replace all rows
        print(f'    Replace N={un[v].sum()}')
        if SEND_TO_DB:
            db.execute(f"""delete from nirspec_phot_match where file_phot = '{v}'""")
            db.send_to_database('nirspec_phot_match', matched_tab[un[v]], if_exists='append')

    elif (new.sum() > 0):
        # Just send new rows
        if SEND_TO_DB:
            db.send_to_database('nirspec_phot_match', matched_tab[new], if_exists='append')

Add   249 rows (   0 in matched_tab,  804 in table)) to nirspec_phot_match for file_phot = 'primer-cosmos-east-grizli-v7.0-fix_phot.fits'
Add   247 rows (   0 in matched_tab, 1646 in table)) to nirspec_phot_match for file_phot = 'primer-cosmos-west-grizli-v7.0-fix_phot.fits'
